In [1]:
import cv2
import cv_bridge
import matplotlib.pyplot as plt
import minkindr
import numpy as np
import rosbag
import rospy
import scipy
from prophesee_event_msgs.msg import Event, EventArray
from geometry_msgs.msg import PoseStamped
from sensor_msgs.msg import CameraInfo, Image, Imu
from std_msgs.msg import *


In [2]:
# add configuration
bridge = cv_bridge.CvBridge()
config={}
config["bag_path"]="/home/mpl/datasets/nfov_day_loopychessboard_filter.bag"
config["event_topic"]="/prophesee/camera/cd_events_buffer"
config["imu_topic"]="/imu"

In [3]:
bag = rosbag.Bag(config["bag_path"], "r")
eva_msgs=[m.message for m in bag.read_messages(config["event_topic"])]
imu_msgs=[m.message for m in bag.read_messages(config["imu_topic"])]

FileNotFoundError: [Errno 2] No such file or directory: 'nfov_day_loopychessboard_filter.bag'

In [4]:
event_array=[]
for ea in eva_msgs:
    for e in ea.events:
        event_array.append(e)

In [10]:
event_np = np.zeros([len(event_array),4])
for i in range(len(event_array)):
    event_np[i,0] = event_array[i].ts.secs+event_array[i].ts.nsecs/1e9
    event_np[i,1] = event_array[i].x
    event_np[i,2] = event_array[i].y
    event_np[i,3] = 1 if event_array[i].polarity else 0
event_np = event_np[event_np[:, 0].argsort()]
np.save("events.npy",event_np)

In [3]:
# event_np=np.load("events.npy")
t_interval = 0.003
t_start = event_np[0,0]
img_buf=[]

count = 0
l = event_np.shape[0]
ace_img=np.zeros([480,640],dtype=np.uint8)
while count < l:
    if event_np[count,0] < t_start+t_interval:
        x=int(event_np[count,1])
        y=int(event_np[count,2])
        ace_img[y,x]=255
    else:
        img_buf.append(ace_img)
        ace_img=np.zeros([480,640],dtype=np.uint8)
        t_start+=t_interval
    count+=1

In [4]:
for (i,im) in enumerate(img_buf):
    cv2.imwrite("./imgs/{:06d}.png".format(i),im)